In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class MaxPoolingLayer(nn.Module):
    def __init__(self, pool_size=2):
        """
        Max Pooling Layer (Custom Implementation).
        :param pool_size: Size of the pooling window.
        """
        super(MaxPoolingLayer, self).__init__()
        self.pool_size = pool_size

    def forward(self, x):
        """
        Forward pass of the max-pooling layer.
        """
        batch_size, channels, height, width = x.size()
        output_height, output_width = height // self.pool_size, width // self.pool_size

        # Initialize output tensor
        pooled_output = torch.zeros((batch_size, channels, output_height, output_width)).to(x.device)

        # Loop over pooling windows
        for i in range(0, height, self.pool_size):
            for j in range(0, width, self.pool_size):
                # Extract the current pooling window
                window = x[:, :, i:i + self.pool_size, j:j + self.pool_size]

                # Compute the maximum value in the window
                max_value = torch.max(window.reshape(batch_size, channels, -1), dim=2)[0]

                # Assign the max value to the corresponding location in the output tensor
                pooled_output[:, :, i // self.pool_size, j // self.pool_size] = max_value

        return pooled_output


In [ ]:
class MaxPoolingCNN(nn.Module):
    def __init__(self, num_classes=10):
        """
        Simple CNN with Max Pooling Layer.
        """
        super(MaxPoolingCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  # conv1 has 32 filters, so it produces 32 output channels
        self.max_pool = MaxPoolingLayer(pool_size=2)
        self.fc1 = nn.Linear(32 * 14 * 14, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # First convolutional layer
        x = self.max_pool(x)       # First max pooling
        x = x.view(x.size(0), -1)  # Flatten for FC layer
        x = self.fc1(x)            # Fully connected layer to output classes
        return x

In [ ]:
class FuzzyPoolingLayer(nn.Module):
    def __init__(self, pool_size=2):
        """
        Fuzzy Pooling Layer
        :param pool_size: Size of pooling window
        """
        super(FuzzyPoolingLayer, self).__init__()
        self.pool_size = pool_size

    def gaussian_membership(self, x, mean, std):
        """
        Compute Gaussian Membership Function (GMF) for input tensor.
        """
        return torch.exp(-((x - mean) ** 2) / (2 * std ** 2))

    def forward(self, x):
        """
        Forward pass of the fuzzy-based pooling layer.
        """
        batch_size, channels, height, width = x.size()
        output_height, output_width = height // self.pool_size, width // self.pool_size

        # Initialize pooled output
        pooled_output = torch.zeros((batch_size, channels, output_height, output_width)).to(x.device)

        # Loop over pooling windows
        for i in range(0, height, self.pool_size):
            for j in range(0, width, self.pool_size):
                # Extract the current pooling window
                window = x[:, :, i:i + self.pool_size, j:j + self.pool_size]

                # Compute mean and standard deviation for the window
                mean = window.mean(dim=(2, 3), keepdim=True)
                std = window.std(dim=(2, 3), keepdim=True) + 1e-8  # Avoid divide-by-zero

                # Compute GMF (Gaussian Membership Function) values for the window
                gmf = self.gaussian_membership(window, mean, std)

                # Threshold: S-norm (max) followed by T-norm (min)
                threshold = torch.min(torch.max(gmf, dim=3)[0], dim=2)[0].unsqueeze(-1).unsqueeze(-1)

                # Identify dominant features (only keep elements above the threshold)
                dominant_features = window * (gmf >= threshold).float()

                # Compute weighted average of the dominant features
                weights = gmf / (gmf.sum(dim=(2, 3), keepdim=True) + 1e-8)
                pooled_value = (dominant_features * weights).sum(dim=(2, 3))

                # Assign the pooled value to the corresponding location in the output tensor
                pooled_output[:, :, i // self.pool_size, j // self.pool_size] = pooled_value

        return pooled_output

In [ ]:
class FuzzyCNN(nn.Module):
    def __init__(self, num_classes=10):
        """
        Simple CNN with Fuzzy-Based Pooling Layer.
        """
        super(FuzzyCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)  ## self.conv1 = nn.Conv2d(in_channels=3 (RGB), out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fuzzy_pool = FuzzyPoolingLayer(pool_size=2)
        self.fc1 = nn.Linear(32 * 14 * 14, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.fuzzy_pool(x)
        x = x.view(x.size(0), -1)  # Flatten for FC layer
        x = self.fc1(x)
        return x

In [ ]:
# Training function with loss & accuracy tracking
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()

    # Lists to store loss and accuracy per epoch for plotting
    epoch_losses = []
    epoch_accuracies = []

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Compute average loss and accuracy for this epoch
        avg_loss = total_loss / len(train_loader)
        accuracy = 100 * correct / total

        # Store values for plotting
        epoch_losses.append(avg_loss)
        epoch_accuracies.append(accuracy)

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")







In [ ]:
# Test function to evaluate the model on test data
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [ ]:
# Helper function to plot loss and accuracy
def plot_metrics(losses, accuracies):
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, losses, '-o', label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracies, '-o', label='Accuracy', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy')
    plt.legend()

    plt.show()

In [ ]:
# Example Usage
if __name__ == "__main__":
    from torchvision.datasets import MNIST
    from torchvision.transforms import ToTensor
    from torch.utils.data import DataLoader

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load dataset
    train_dataset = MNIST(root='data/', train=True, transform=ToTensor(), download=True)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    test_dataset = MNIST(root='data/', train=False, transform=ToTensor(), download=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



In [ ]:
    # Initialize model, loss, and optimizer
    modelMax = MaxPoolingCNN(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modelMax.parameters(), lr=0.001)

    # Train the model
    train(modelMax, train_loader, criterion, optimizer, epochs=5)

Epoch [1/5], Loss: 0.2355, Accuracy: 93.22%
Epoch [2/5], Loss: 0.0869, Accuracy: 97.46%
Epoch [3/5], Loss: 0.0650, Accuracy: 98.12%
Epoch [4/5], Loss: 0.0533, Accuracy: 98.44%
Epoch [5/5], Loss: 0.0460, Accuracy: 98.66%


In [ ]:
    # Evaluate the model on test data
    test_loss, test_accuracy = test(modelMax, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.0549, Test Accuracy: 98.12%


In [ ]:
    # Initialize model, loss, and optimizer
    modelFuzzy = FuzzyCNN(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modelFuzzy.parameters(), lr=0.001)

    # Train the model
    train(modelFuzzy, train_loader, criterion, optimizer, epochs=5)

Epoch [1/5], Loss: 0.3223, Accuracy: 91.05%
Epoch [2/5], Loss: 0.1308, Accuracy: 96.29%
Epoch [3/5], Loss: 0.0918, Accuracy: 97.39%
Epoch [4/5], Loss: 0.0728, Accuracy: 97.88%
Epoch [5/5], Loss: 0.0624, Accuracy: 98.22%


In [ ]:
    # Evaluate the model on test data
    test_loss, test_accuracy = test(modelFuzzy, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.0674, Test Accuracy: 97.85%


In [ ]:
class AveragePoolingLayer(nn.Module):
    def __init__(self, pool_size=2):
        """
        Average Pooling Layer
        :param pool_size: Size of pooling window
        """
        super(AveragePoolingLayer, self).__init__()
        self.pool = nn.AvgPool2d(kernel_size=pool_size, stride=pool_size)

    def forward(self, x):
        return self.pool(x)


class CNNWithAveragePooling(nn.Module):
    def __init__(self, num_classes=10):
        """
        Simple CNN with Average Pooling Layer.
        """
        super(CNNWithAveragePooling, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.avg_pool = AveragePoolingLayer(pool_size=2)
        self.fc1 = nn.Linear(32 * 14 * 14, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten for FC layer
        x = self.fc1(x)
        return x

In [ ]:
    # Initialize model, loss, and optimizer
    modelAvg = CNNWithAveragePooling(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modelAvg.parameters(), lr=0.001)

    # Train the model
    train(modelAvg, train_loader, criterion, optimizer, epochs=5)

Epoch [1/5], Loss: 0.2835, Accuracy: 92.03%
Epoch [2/5], Loss: 0.1069, Accuracy: 97.04%
Epoch [3/5], Loss: 0.0758, Accuracy: 97.84%
Epoch [4/5], Loss: 0.0632, Accuracy: 98.19%
Epoch [5/5], Loss: 0.0530, Accuracy: 98.48%


In [ ]:
    # Evaluate the model on test data
    test_loss, test_accuracy = test(modelAvg, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.0541, Test Accuracy: 98.26%


In [ ]:
class FuzzyPoolingLayer(nn.Module):
    def __init__(self, pool_size=2, reg_lambda=0.01):
        """
        Fuzzy Pooling Layer with RegP (Regularization by Pooling).
        :param pool_size: Size of pooling window
        :param reg_lambda: Regularization factor to control sparsity.
        """
        super(FuzzyPoolingLayer, self).__init__()
        self.pool_size = pool_size
        self.reg_lambda = reg_lambda  # Controls regularization strength

    def gaussian_membership(self, x, mean, std):
        """
        Compute Gaussian Membership Function (GMF) for input tensor.
        """
        return torch.exp(-((x - mean) ** 2) / (2 * std ** 2))

    def regularization_penalty(self, gmf):
        """
        RegP: Compute the regularization penalty for GMF activations.
        This penalty encourages sparse activations to reduce redundancy.
        """
        # L2 norm regularization over the GMF values to penalize dense activations
        penalty = torch.sum(gmf ** 2, dim=(2, 3))  # Sum over spatial dimensions
        return self.reg_lambda * penalty.mean()  # Apply lambda and take mean

    def forward(self, x):
        """
        Forward pass of the fuzzy-based pooling layer with RegP.
        """
        batch_size, channels, height, width = x.size()
        output_height, output_width = height // self.pool_size, width // self.pool_size

        # Initialize pooled output
        pooled_output = torch.zeros((batch_size, channels, output_height, output_width)).to(x.device)
        regp_total = 0  # Store the total RegP penalty

        # Loop over pooling windows
        for i in range(0, height, self.pool_size):
            for j in range(0, width, self.pool_size):
                # Extract the current pooling window
                window = x[:, :, i:i + self.pool_size, j:j + self.pool_size]

                # Compute mean and standard deviation for the window
                mean = window.mean(dim=(2, 3), keepdim=True)
                std = window.std(dim=(2, 3), keepdim=True) + 1e-8  # Avoid divide-by-zero

                # Compute GMF (Gaussian Membership Function) values for the window
                gmf = self.gaussian_membership(window, mean, std)

                # Apply RegP to penalize dense activations
                regp_total += self.regularization_penalty(gmf)

                # Threshold: S-norm (max) followed by T-norm (min)
                threshold = torch.min(torch.max(gmf, dim=3)[0], dim=2)[0].unsqueeze(-1).unsqueeze(-1)

                # Identify dominant features (only keep elements above the threshold)
                dominant_features = window * (gmf >= threshold).float()

                # Compute weighted average of the dominant features
                weights = gmf / (gmf.sum(dim=(2, 3), keepdim=True) + 1e-8)
                pooled_value = (dominant_features * weights).sum(dim=(2, 3))

                # Assign the pooled value to the corresponding location in the output tensor
                pooled_output[:, :, i // self.pool_size, j // self.pool_size] = pooled_value

        # Add RegP penalty to the model as a regularization loss
        self.regp_loss = regp_total / (batch_size * channels)  # Normalize by batch size and channels
        return pooled_output




In [ ]:
class FuzzyCNNWithRegP(nn.Module):
    def __init__(self, num_classes=10):
        """
        Simple CNN with Fuzzy-Based Pooling Layer and RegP.
        """
        super(FuzzyCNNWithRegP, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.fuzzy_pool = FuzzyPoolingLayer(pool_size=2, reg_lambda=0.01)
        self.fc1 = nn.Linear(32 * 14 * 14, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.fuzzy_pool(x)
        x = x.view(x.size(0), -1)  # Flatten for FC layer
        x = self.fc1(x)
        return x

    def get_regp_loss(self):
        """
        Retrieve the RegP loss from the pooling layer.
        """
        return self.fuzzy_pool.regp_loss

In [ ]:
    # Initialize model, loss, and optimizer
    modelR = FuzzyCNNWithRegP(num_classes=10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(modelR.parameters(), lr=0.001)

    # Train the model
    train(modelR, train_loader, criterion, optimizer, epochs=5)

Epoch [1/5], Loss: 0.3376, Accuracy: 90.60%
Epoch [2/5], Loss: 0.1423, Accuracy: 95.97%
Epoch [3/5], Loss: 0.0975, Accuracy: 97.17%


In [ ]:
    # Evaluate the model on test data
    test_loss, test_accuracy = test(modelR, test_loader, criterion, device)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

NameError: name 'test' is not defined

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary

def model_summary(model, input_size=(1, 28, 28)):
    """
    Print a summary of the model, including layers, output shapes, and parameter counts.

    :param model: PyTorch model instance
    :param input_size: Input shape for the model (excluding batch size)
    """
    print("----- Model Summary -----")
    print(model)
    print("\n----- Layer-wise Details -----")
    summary(model, input_size=input_size)

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print("\n----- Parameter Details -----")
    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    print(f"Non-trainable Parameters: {total_params - trainable_params:,}")

    # If RegP is present, provide its status.
    if hasattr(model, 'get_regp_loss'):
        print("\nRegP Enabled: Yes")
    else:
        print("\nRegP Enabled: No")

In [ ]:
model_summary(modelMax, input_size=(1, 28, 28))

In [ ]:
model_summary(modelFuzzy, input_size=(1, 28, 28))

In [ ]:
model_summary(modelR, input_size=(1, 28, 28))

In [ ]:
model_summary(modelAvg, input_size=(1, 28, 28))